# 2023 CMAQ-CRACMM3 Project: PM2.5, O3, and Photochemical Age Analysis
---
    Updated paths for local analysis
    Added O3 analysis alongside PM2.5
    Data location: D:\Raw_Data\CMAQ_Model
---

In [ ]:
%matplotlib inline

## Import Libraries

In [ ]:
# Libraries and packages
import cmaqsatproc as csp
import pandas as pd
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import cartopy.feature
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
import numpy as np
import glob
import netCDF4
from netCDF4 import Dataset
import PseudoNetCDF as pnc
from datetime import datetime, timedelta, date
import warnings
import scipy
from scipy.stats import bootstrap
import math
from IPython.display import display
import ast
import adjustText 
import textalloc as ta
import plotly
import plotly.express as px
import nbformat
from scipy.stats import gaussian_kde

import pyrsig
import pycno
from dateutil import rrule
import cmaps

In [ ]:
# Print versions
print('csp version:    ' + csp.__version__)
print('pandas version: ' + pd.__version__)
print('xr version:     ' + xr.__version__)
print('cartopy version:' + cartopy.__version__)
print('Numpy version:  ' + np.__version__)
print('Netcdf4 version:' + netCDF4.__version__)
print('pnc version:    ' + pnc.__version__)
print('pyrsig version: ' + pyrsig.__version__)
print('pyncno version: ' + pycno.__version__)

In [ ]:
today = str(date.today())
today = today.replace('-', '')
print('Last run on YYYYMMDD ' + str(today))

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
# Set verbose to true to make more documentation
verbose = False

## Setup - Updated Paths for D:\Raw_Data

In [ ]:
# Major file locations - UPDATED PATHS
basedir = 'D:/Raw_Data/CMAQ_Model/'
datadir = basedir  # Input data and cities.txt
outdir = basedir + 'outputs/'  # For saving CSV output and plots

# Create output directory if it doesn't exist
import os
os.makedirs(outdir, exist_ok=True)
print(f'Base directory: {basedir}')
print(f'Output directory: {outdir}')

In [ ]:
# Proj4 info and lat/lon info
metpath = basedir + 'MOD3DATA_MET/'
m2dpath = metpath + 'METCRO2D.12US4.35L.230701'
g2dpath = metpath + 'GRIDCRO2D.12US4.35L.230701'

print(f'Meteorology path: {metpath}')
print(f'METCRO2D: {m2dpath}')
print(f'GRIDCRO2D: {g2dpath}')

In [ ]:
# Display the proj4 string
f = pyrsig.open_ioapi(m2dpath)  
f.crs_proj4
cno = pycno.cno(f.crs_proj4)

In [ ]:
def set_map(ax, title, colorlabel):
    """Helper function to format map plots"""
    ax.colorbar.set_label(colorlabel)
    ax.axes.set_xlabel('')
    ax.axes.set_xticklabels('')
    ax.axes.set_ylabel('')
    ax.axes.set_yticklabels('')
    plt.setp(ax.axes, **dict(title=title))
    cno.drawstates(ax=ax.axes, linewidth=0.2)

## Load CMAQ Data - CMAQv6a1 CRACMM3

In [ ]:
# CMAQv6a1 CRACMM3 - Updated paths
cmaqbasedata = basedir + 'netcdffiles/COMBINE_ACONC_cmaq6acracmm3_base_2023_12US4_202306.nc'
cmaqnofiredata = basedir + 'netcdffiles/COMBINE_ACONC_cmaq6acracmm3_nofire_2023_12US4_202306.nc'

print('Loading CMAQ data...')
print(f'Base scenario: {cmaqbasedata}')
print(f'No-fire scenario: {cmaqnofiredata}')

baseconc = pyrsig.open_ioapi(cmaqbasedata)    
nofireconc = pyrsig.open_ioapi(cmaqnofiredata)

print('Data loaded successfully!')

## Calculate Fire Impacts (Base - NoFire)

In [ ]:
# Mean changes (2-D mean in time)
print('Calculating fire impacts...')

# PM2.5
pm25_ugm3 = baseconc['PM25_TOT'].mean(dim='TSTEP') - nofireconc['PM25_TOT'].mean(dim='TSTEP')

# Ozone - NEW!
o3_ppb = baseconc['O3'].mean(dim='TSTEP') - nofireconc['O3'].mean(dim='TSTEP')

# CO
co_ppb = baseconc['CO'].mean(dim='TSTEP') - nofireconc['CO'].mean(dim='TSTEP')
co_ugm3 = ((baseconc['CO'] - nofireconc['CO']) * baseconc['AIR_DENS'] * 28.01 / 28.9628).mean(dim='TSTEP')

# Benzene and Toluene for age calculation
benzene_ppb = baseconc['BENZENE'].mean(dim='TSTEP') - nofireconc['BENZENE'].mean(dim='TSTEP')
toluene_ppb = baseconc['TOLUENE'].mean(dim='TSTEP') - nofireconc['TOLUENE'].mean(dim='TSTEP')

print('Fire impact calculations complete!')

## Calculate Photochemical Age

In [ ]:
# Calculate photochemical age (2-D mean in time)
bentotolinitial = 2.27  # ptothna input, emission molar ratio
bentoltime = (benzene_ppb) / (toluene_ppb)
bentoltime = np.log(bentoltime / bentotolinitial) / (5.9337e-12 - 1.2196e-12) / 3600 / 24 / 1e6

print('Photochemical age calculated')
print(f'Age range: {bentoltime.min().values:.2f} to {bentoltime.max().values:.2f} days')

## PM2.5 Concentration Maps

In [ ]:
spc = 'PM25_TOT'
units = 'μg/m³'
printname = '$PM_{2.5}$'
colmap = plt.get_cmap('YlOrRd')
          
# Create 4-panel plot
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['figure.figsize'] = [14, 1.8]
f1, sax = plt.subplots(1, 4, dpi=300)
plt.subplots_adjust(wspace=0.3)

# Panel 0: Base simulation
plt.sca(sax[0])
conc = baseconc[spc].mean(dim='TSTEP')
print(f"PM2.5 base min, max: {conc.min().values:.2f}, {conc.max().values:.2f}")
levelsforplot = [0, 10, 20, 30, 40, 50, 60, 70, 80]
plotvar = conc.plot(cmap='viridis', levels=levelsforplot)
title = f'June {printname}'
sax[0].axes.set_xticks([])
sax[0].axes.set_yticks([])
set_map(ax=plotvar, title=title, colorlabel=units)

# Panel 1: No-fire simulation
plt.sca(sax[1])
conc = nofireconc[spc].mean(dim='TSTEP')
print(f"PM2.5 no fire min, max: {conc.min().values:.2f}, {conc.max().values:.2f}")
levelsforplot = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
plotvar = conc.plot(cmap='viridis', levels=levelsforplot)
title = f'No fire (Max: {conc.max().values:.0f}, Min: {conc.min().values:.0f})'
sax[1].axes.set_xticks([])
sax[1].axes.set_yticks([])
set_map(ax=plotvar, title=title, colorlabel=units)

# Panel 2: Fire delta (absolute)
plt.sca(sax[2])
conc = pm25_ugm3
print(f"PM2.5 delta min, max: {conc.min().values:.2f}, {conc.max().values:.2f}")
levelsforplot = [-0.5, 0, 0.5, 1, 2, 3, 4, 5, 10, 20]
plotvar = conc.plot(cmap=colmap, levels=levelsforplot)
title = f'$\\Delta$$C_i$ (Max: {conc.max().values:.1f}, Min: {conc.min().values:.1e})'
sax[2].axes.set_xticks([])
sax[2].axes.set_yticks([])
set_map(ax=plotvar, title=title, colorlabel=units)

# Panel 3: Fire delta (percent)
plt.sca(sax[3])
conc = (baseconc[spc].mean(dim='TSTEP') - nofireconc[spc].mean(dim='TSTEP')) / baseconc[spc].mean(dim='TSTEP') * 100
print(f"PM2.5 delta % min, max: {conc.min().values:.2f}, {conc.max().values:.2f}")
levelsforplot = [-1, 0, 1, 2, 3, 4, 5, 10, 50, 100]
plotvar = conc.plot(cmap=colmap, levels=levelsforplot)
title = f'%$\\Delta$$C_i$ (Max: {conc.max().values:.1f}, Min: {conc.min().values:.1e})'
sax[3].axes.set_xticks([])
sax[3].axes.set_yticks([])
set_map(ax=plotvar, title=title, colorlabel='%')

plt.savefig(f'{outdir}/{today}_PM25_maps.png', dpi=300, bbox_inches='tight')
plt.show()

## O3 Concentration Maps - NEW ANALYSIS

In [ ]:
spc = 'O3'
units = 'ppb'
printname = '$O_3$'
colmap_o3 = plt.get_cmap('RdBu_r')  # Red-Blue reversed for O3
          
# Create 4-panel plot
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['figure.figsize'] = [14, 1.8]
f1, sax = plt.subplots(1, 4, dpi=300)
plt.subplots_adjust(wspace=0.3)

# Panel 0: Base simulation
plt.sca(sax[0])
conc = baseconc[spc].mean(dim='TSTEP')
print(f"O3 base min, max: {conc.min().values:.2f}, {conc.max().values:.2f}")
levelsforplot = [0, 10, 20, 30, 40, 50, 60, 70, 80]
plotvar = conc.plot(cmap='viridis', levels=levelsforplot)
title = f'June {printname}'
sax[0].axes.set_xticks([])
sax[0].axes.set_yticks([])
set_map(ax=plotvar, title=title, colorlabel=units)

# Panel 1: No-fire simulation
plt.sca(sax[1])
conc = nofireconc[spc].mean(dim='TSTEP')
print(f"O3 no fire min, max: {conc.min().values:.2f}, {conc.max().values:.2f}")
levelsforplot = [0, 10, 20, 30, 40, 50, 60, 70, 80]
plotvar = conc.plot(cmap='viridis', levels=levelsforplot)
title = f'No fire (Max: {conc.max().values:.0f}, Min: {conc.min().values:.0f})'
sax[1].axes.set_xticks([])
sax[1].axes.set_yticks([])
set_map(ax=plotvar, title=title, colorlabel=units)

# Panel 2: Fire delta (absolute) - Can be positive or negative for O3!
plt.sca(sax[2])
conc = o3_ppb
print(f"O3 delta min, max: {conc.min().values:.2f}, {conc.max().values:.2f}")
levelsforplot = [-10, -5, -2, -1, 0, 1, 2, 5, 10, 20]
plotvar = conc.plot(cmap=colmap_o3, levels=levelsforplot)
title = f'$\\Delta$$O_3$ (Max: {conc.max().values:.1f}, Min: {conc.min().values:.1f})'
sax[2].axes.set_xticks([])
sax[2].axes.set_yticks([])
set_map(ax=plotvar, title=title, colorlabel=units)

# Panel 3: Fire delta (percent)
plt.sca(sax[3])
conc = (baseconc[spc].mean(dim='TSTEP') - nofireconc[spc].mean(dim='TSTEP')) / nofireconc[spc].mean(dim='TSTEP') * 100
print(f"O3 delta % min, max: {conc.min().values:.2f}, {conc.max().values:.2f}")
levelsforplot = [-20, -10, -5, -2, 0, 2, 5, 10, 20, 50]
plotvar = conc.plot(cmap=colmap_o3, levels=levelsforplot)
title = f'%$\\Delta$$O_3$ (Max: {conc.max().values:.1f}, Min: {conc.min().values:.1f})'
sax[3].axes.set_xticks([])
sax[3].axes.set_yticks([])
set_map(ax=plotvar, title=title, colorlabel='%')

plt.savefig(f'{outdir}/{today}_O3_maps.png', dpi=300, bbox_inches='tight')
plt.show()

## Photochemical Age Map

In [ ]:
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['figure.figsize'] = [3.3, 2.25]
f1, sax = plt.subplots(1, 1, dpi=300)
plt.sca(sax)

conc = bentoltime
lab = 'days'
colmap = plt.get_cmap('Purples')
levelsforplot = [0, 0.5, 1, 2, 3, 4, 5, 6]
plotvar = conc.plot(cmap=colmap, levels=levelsforplot, edgecolor=None)
sax.axes.set_xticks([])
sax.axes.set_yticks([])
title = 'Photochemical age (days)'
set_map(ax=plotvar, title=title, colorlabel=lab)

plt.savefig(f'{outdir}/{today}_photochem_age_map.png', dpi=300, bbox_inches='tight')
plt.show()

## Load Cities for Time Series Analysis

In [ ]:
# Load grid info
grid = pnc.pncopen(g2dpath, format='ioapi')

# Load cities
sites = pd.read_csv(datadir + 'cities.txt', header=None)
sites = sites.rename(columns={1: "lon", 2: "lat", 0: 'site'})
print('Cities loaded:')
display(sites)

## Prepare Data for Smoke-Masked Analysis

In [ ]:
# Mask for smoke conditions
smokemask = benzene_ppb > 0.010  # 10 ppt benzene enhancement to detect fire smoke

# Create 1-D data (masked for smoke)
time_masked = bentoltime.where(smokemask, drop=True).squeeze().stack(z=('ROW', 'COL')).dropna(dim="z", how="any")
coppb_masked = co_ppb.where(smokemask, drop=True).squeeze().stack(z=('ROW', 'COL')).dropna(dim="z", how="any")
pm25ugm3_masked = pm25_ugm3.where(smokemask, drop=True).squeeze().stack(z=('ROW', 'COL')).dropna(dim="z", how="any")
o3ppb_masked = o3_ppb.where(smokemask, drop=True).squeeze().stack(z=('ROW', 'COL')).dropna(dim="z", how="any")  # NEW!

# PM2.5/CO and O3/CO ratios
pmrelcogg_masked = (pm25_ugm3.squeeze() / co_ugm3.squeeze()).where(smokemask.squeeze(), drop=True).stack(z=('ROW', 'COL')).dropna(dim="z", how="any")
o3relco_masked = (o3_ppb.squeeze() / co_ppb.squeeze()).where(smokemask.squeeze(), drop=True).stack(z=('ROW', 'COL')).dropna(dim="z", how="any")  # NEW!

# Create DataFrame
dfplot = pd.DataFrame({
    'time': time_masked,
    'co': coppb_masked,
    'pm25': pm25ugm3_masked,
    'o3': o3ppb_masked,
    'pmco_gg': pmrelcogg_masked,
    'o3co': o3relco_masked
})

print(f'Smoke-masked data points: {len(dfplot)}')
print('\nData summary:')
display(dfplot.describe())

In [ ]:
# Create quantile bins
qcount = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
dfplot['bins'] = pd.qcut(np.log(dfplot['time']), qcount)
dfbin = dfplot.groupby(['bins']).median().dropna()

print('Binned data created')
display(dfbin)

## PM2.5 and O3 vs Photochemical Age - COMBINED ANALYSIS

In [ ]:
panelfontsize = 10
fig, rax = plt.subplots(
    4, 1, figsize=(3.3, 13.2), gridspec_kw=dict(
        wspace=0.35, left=0.06, right=0.99, top=0.95, bottom=0.15
    ), dpi=300
)

# Panel 0: PM2.5 vs age
ax = rax[0]
ax.plot(dfbin['time'], dfbin['pm25'], '--', color='grey')
ax.plot(dfbin['time'], dfbin['pm25'], '.', color='grey')
ax.set(xlabel='', ylabel='June $\\Delta$ PM$_{2.5}$ ($\\mu$g$m^{-3}$)', yscale='log')
ax.xaxis.set_tick_params(labelsize=panelfontsize)
ax.yaxis.set_tick_params(labelsize=panelfontsize)
ax.xaxis.label.set_size(panelfontsize)
ax.yaxis.label.set_size(panelfontsize)

# Add city markers
for index, row in sites.iterrows():
    lonin = row['lon']
    latin = row['lat']
    i, j = grid.ll2ij(lonin, latin)
    yval = np.array((pm25_ugm3.squeeze())[j, i].squeeze().data, ndmin=1)[0]
    xval = np.array(bentoltime[:, j, i].squeeze().data, ndmin=1)[0]
    ax.text(xval, yval, row['site'], fontsize=6)
    ax.scatter(xval, yval, marker='.', color='black')

ax.set_title('(a) PM$_{2.5}$ vs Age')
ax.set_xlim(0, 5)
ax.set_ylim(0.1, 100)

# Panel 1: PM2.5/CO vs age
ax = rax[1]
ax.plot(dfbin['time'], dfbin['pmco_gg'], '--', color='grey')
ax.plot(dfbin['time'], dfbin['pmco_gg'], '.', color='grey')
ax.set(xlabel='', ylabel='June $\\Delta$ PM$_{2.5}$/$\\Delta$CO (g/g)')
ax.xaxis.set_tick_params(labelsize=panelfontsize)
ax.yaxis.set_tick_params(labelsize=panelfontsize)
ax.xaxis.label.set_size(panelfontsize)
ax.yaxis.label.set_size(panelfontsize)

# Add city markers
for index, row in sites.iterrows():
    lonin = row['lon']
    latin = row['lat']
    i, j = grid.ll2ij(lonin, latin)
    yval = np.array((pm25_ugm3.squeeze() / co_ugm3.squeeze())[j, i].data, ndmin=1)[0]
    xval = np.array(bentoltime[:, j, i].squeeze().data, ndmin=1)[0]
    ax.text(xval, yval, row['site'], fontsize=6)
    ax.scatter(xval, yval, marker='.', color='black')

ax.set_title('(b) PM$_{2.5}$/CO vs Age')
ax.set_xlim(0, 5)

# Panel 2: O3 vs age - NEW!
ax = rax[2]
ax.plot(dfbin['time'], dfbin['o3'], '--', color='blue')
ax.plot(dfbin['time'], dfbin['o3'], '.', color='blue')
ax.set(xlabel='', ylabel='June $\\Delta$ O$_3$ (ppb)')
ax.axhline(y=0, color='red', linestyle=':', linewidth=1)  # Zero line
ax.xaxis.set_tick_params(labelsize=panelfontsize)
ax.yaxis.set_tick_params(labelsize=panelfontsize)
ax.xaxis.label.set_size(panelfontsize)
ax.yaxis.label.set_size(panelfontsize)

# Add city markers
for index, row in sites.iterrows():
    lonin = row['lon']
    latin = row['lat']
    i, j = grid.ll2ij(lonin, latin)
    yval = np.array((o3_ppb.squeeze())[j, i].squeeze().data, ndmin=1)[0]
    xval = np.array(bentoltime[:, j, i].squeeze().data, ndmin=1)[0]
    ax.text(xval, yval, row['site'], fontsize=6)
    ax.scatter(xval, yval, marker='.', color='black')

ax.set_title('(c) O$_3$ vs Age')
ax.set_xlim(0, 5)

# Panel 3: O3/CO vs age - NEW!
ax = rax[3]
ax.plot(dfbin['time'], dfbin['o3co'], '--', color='blue')
ax.plot(dfbin['time'], dfbin['o3co'], '.', color='blue')
ax.set(xlabel='Photochemical age (days)', ylabel='June $\\Delta$ O$_3$/$\\Delta$CO (ppb/ppb)')
ax.axhline(y=0, color='red', linestyle=':', linewidth=1)  # Zero line
ax.xaxis.set_tick_params(labelsize=panelfontsize)
ax.yaxis.set_tick_params(labelsize=panelfontsize)
ax.xaxis.label.set_size(panelfontsize)
ax.yaxis.label.set_size(panelfontsize)

# Add city markers
for index, row in sites.iterrows():
    lonin = row['lon']
    latin = row['lat']
    i, j = grid.ll2ij(lonin, latin)
    yval = np.array((o3_ppb.squeeze() / co_ppb.squeeze())[j, i].data, ndmin=1)[0]
    xval = np.array(bentoltime[:, j, i].squeeze().data, ndmin=1)[0]
    ax.text(xval, yval, row['site'], fontsize=6)
    ax.scatter(xval, yval, marker='.', color='black')

ax.set_title('(d) O$_3$/CO vs Age')
ax.set_xlim(0, 5)

plt.savefig(f'{outdir}/{today}_PM25_O3_vs_age.png', dpi=300, bbox_inches='tight')
plt.show()

## Summary Statistics

In [ ]:
print('=== SUMMARY STATISTICS ===')
print('\nPM2.5 Fire Impact:')
print(f'  Mean: {pm25_ugm3.mean().values:.2f} μg/m³')
print(f'  Max:  {pm25_ugm3.max().values:.2f} μg/m³')
print(f'  Min:  {pm25_ugm3.min().values:.2f} μg/m³')

print('\nO3 Fire Impact:')
print(f'  Mean: {o3_ppb.mean().values:.2f} ppb')
print(f'  Max:  {o3_ppb.max().values:.2f} ppb')
print(f'  Min:  {o3_ppb.min().values:.2f} ppb')

print('\nPhotochemical Age:')
print(f'  Mean: {bentoltime.mean().values:.2f} days')
print(f'  Max:  {bentoltime.max().values:.2f} days')

print('\nCO Fire Impact:')
print(f'  Mean: {co_ppb.mean().values:.2f} ppb')
print(f'  Max:  {co_ppb.max().values:.2f} ppb')

## Export Data to CSV

In [ ]:
# Save the smoke-masked data
csv_file = f'{outdir}/{today}_smoke_masked_data.csv'
dfplot.to_csv(csv_file, index=False)
print(f'Saved smoke-masked data to: {csv_file}')

# Save binned data
csv_file_bin = f'{outdir}/{today}_binned_data.csv'
dfbin.to_csv(csv_file_bin)
print(f'Saved binned data to: {csv_file_bin}')

## Analysis Complete!

This notebook analyzed:
- ✓ PM2.5 fire impacts (concentrations, maps, time series)
- ✓ O3 fire impacts (concentrations, maps, time series) - NEW!
- ✓ Photochemical age from benzene/toluene ratios
- ✓ PM2.5/CO and O3/CO ratios vs age
- ✓ City-specific impacts

All outputs saved to: `D:/Raw_Data/CMAQ_Model/outputs/`